# Progressive GAN

In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize, Lambda
from torch.utils.data import DataLoader

# Data

In [70]:
def transform_in_size(size):
    return Compose([Resize((size, size)), ToTensor(), Lambda(lambda x: x * 2 - 1)])

def get_dataloader(size):
    dataset = CIFAR10('~/pytorch', train=True, download=True, transform=transform_in_size(size))
    return DataLoader(dataset, batch_size=32, drop_last=True)

# CUDA

In [71]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
    
print(f"Learning on: {device}")

Learning on: cuda


# Generator
* TODO: Change UpsamplingBilinear to interpolate()
* TODO: Move to inner class

In [72]:
class GeneratorBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.upsample = nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        
    def forward(self, x):
        _, x = self.forward_with_img(x)
        
        return x
    
    def forward_with_img(self, x):
        img = self.upsample(x)
        x = F.leaky_relu(self.conv1(img))
        x = F.leaky_relu(self.conv2(x))
        
        return img, x

In [73]:
class Generator(nn.Module):
    def __init__(self, channels=512):
        super().__init__()
        self.channels = channels
        self.trained_blocks = nn.ModuleList()
        self.new_block = self.create_initial_layer(channels)
        self.rgb = self.to_rgb(channels)
        
    def to_rgb(self, in_channels):
        return nn.Conv2d(in_channels, 3, 1, padding=0)
    
    def create_initial_layer(self, channels):
        module = nn.Sequential(
            nn.Conv2d(channels, channels, 4, padding=3),
            nn.LeakyReLU(),
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.LeakyReLU())
        
        return module
    
    def append_layer(self, channels):
        self.trained_blocks.append(self.new_block)
        self.new_block = GeneratorBlock(self.channels, channels)
        self.rgb = self.to_rgb(channels)
        self.channels = channels
        
    def forward(self, x, alpha=1.0):
        for block in self.trained_blocks:
            x = block(x)
            
        if alpha < 1.0:
            img , x = self.new_block.forward_with_img(x)
            img = self.rgb(img)
            x = self.rgb(x)
            
            x = img * (1 - alpha) + img * alpha
        else:
            x = self.new_block(x)
            x = self.rgb(x)
        
        return x

# Disciminator

In [74]:
class DiscriminatorBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.downsample = nn.UpsamplingBilinear2d(scale_factor=0.5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.downsample(x)
        
        return x

In [75]:
class Discriminator(nn.Module):
    def __init__(self, channels=512):
        super().__init__()
        self.channels = channels
        self.trained_layers = nn.ModuleList()
        self.new_layer = self.create_initial_layer(channels)
        self.downsample = nn.UpsamplingBilinear2d(scale_factor=0.5)
        self.rgb = self.from_rgb(channels)
        
    def create_initial_layer(self, channels):
        module = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.Conv2d(channels, channels, 4, padding=0),
            nn.Flatten(),
            nn.Linear(channels, 1))
        
        return module
    
    def prepend_layer(self, channels):
        self.trained_layers.insert(0, self.new_layer)
        self.new_layer = DiscriminatorBlock(channels, self.channels)
        self.rgb = self.from_rgb(channels)
        
    def from_rgb(self, channels):
        return nn.Conv2d(3, channels, 1, padding=0)
        
    def forward(self, x, alpha=1.0):
        if alpha < 1.0:
            skip = self.downsample(x)
            skip = self.rgb(skip)

            x = self.rgb(x)
            x = self.new_layer(x)
            x = skip * (1-alpha) + x * alpha
        else:
            x = self.rgb(x)
            x = self.new_layer(x)
        
        for layer in self.trained_layers:
            x = layer(x)
        
        return x

In [76]:
def train_batch():
    z = torch.randn(32, 512, 1, 1)
    gen_img = g(z)
    print(f"Image size: {gen_img.shape}")
    out = d(gen_img)
    
    print(out.shape)

In [80]:
g = Generator().to(device)
d = Discriminator().to(device)
d_optim = Adam(d.parameters())
g_optim = Adam(g.parameters())

In [81]:
def train_epoch(size):
    d_criterion = nn.BCEWithLogitsLoss()

    
    dl = get_dataloader(size)
    real = torch.ones((32, 1)).to(device)
    fake = torch.zeros((32, 1)).to(device)
    g_loss_total = 0
    d_loss_total = 0
    it = 0
    for real_img, y in dl:
        # print(real_img.shape)
        real_img = real_img.to(device)
        out = d(real_img)
        real_loss = d_criterion(out, real)
        fake_img = g(torch.randn((32, 512, 1, 1), device=device))
        out = d(fake_img)
        fake_loss = d_criterion(out, fake)
        d_loss = 0.5 * (real_loss + fake_loss)
        
        d_optim.zero_grad()
        d_loss.backward()
        d_optim.step()
        
        fake_img = g(torch.randn((32, 512, 1, 1), device=device))
        out = d(fake_img)
        g_loss = d_criterion(out, real)
        
        g_optim.zero_grad()
        g_loss.backward()
        g_optim.step()
        
        it += 1
        g_loss_total += g_loss.item()
        d_loss_total += d_loss.item()
        if it % 100 == 0:
            print(f"G_LOSS: {g_loss_total / it:.4f} - D_LOSS: {d_loss_total / it:.4f}")
        

In [82]:
train_epoch(4)

Files already downloaded and verified
G_LOSS: 2.4969 - D_LOSS: 1.6626
G_LOSS: 4.3406 - D_LOSS: 1.6417
G_LOSS: 3.1742 - D_LOSS: 1.3454
G_LOSS: 2.6005 - D_LOSS: 1.1955
G_LOSS: 2.2438 - D_LOSS: 1.0974
G_LOSS: 2.0270 - D_LOSS: 1.0391
G_LOSS: 1.8544 - D_LOSS: 0.9929
G_LOSS: 1.7207 - D_LOSS: 0.9592
G_LOSS: 1.6572 - D_LOSS: 0.9317
G_LOSS: 1.5696 - D_LOSS: 0.9088
G_LOSS: 1.5003 - D_LOSS: 0.8894
G_LOSS: 1.4358 - D_LOSS: 0.8727
G_LOSS: 1.3884 - D_LOSS: 0.8585
G_LOSS: 1.3431 - D_LOSS: 0.8479
G_LOSS: 1.3016 - D_LOSS: 0.8369
